In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# 실제 train data를 변환하기 - 데이터 읽어들이기

import json
import os
import pandas as pd
import numpy as np

base_dir = os.getcwd()

train_images_path = base_dir + '/data/train/images/'
train_labels_json_path = base_dir + '/data/train/labels_json/'
train_labels_path = base_dir + '/data/train/labels/'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_json_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)

!unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/train_v4.zip' -d '/content/data/'

train_images_list = os.listdir(train_images_path)
train_labels_json_list = os.listdir(train_labels_json_path)

train_images_list.sort()
train_labels_json_list.sort()

In [3]:
# json 라벨 삭제 및 라벨링 확인

import shutil

shutil.rmtree('/content/data/train/labels_json')
shutil.rmtree('/content/data/val/labels_json')

FileNotFoundError: ignored

In [4]:
# 분리한 데이터 이미지와 라벨 파일 일치 여부 확인

print(len(os.listdir('/content/data/train/images/')))
print(len(os.listdir('/content/data/train/labels/')))

print(len(os.listdir('/content/data/val/images/')))
print(len(os.listdir('/content/data/val/labels/')))

train_images_list = os.listdir('/content/data/train/images/')
train_labels_list = os.listdir('/content/data/train/labels/')

train_images_list.sort()
train_labels_list.sort()

for i in range(len(train_images_list)):
  if train_images_list[i].replace('.jpg', '') != train_labels_list[i].replace('.txt', ''):
    print(train_images_list[i])

val_images_list = os.listdir('/content/data/val/images/')
val_labels_list = os.listdir('/content/data/val/labels/')

val_images_list.sort()
val_labels_list.sort()

for i in range(len(val_images_list)):
  if val_images_list[i].replace('.jpg', '') != val_labels_list[i].replace('.txt', ''):
    print(val_images_list[i])


20946
20946
2416
2416


In [5]:
# YOLO 학습을 위한 yaml 파일 생성

!pip install PyYAML

import yaml

data = {
    'train': '/content/data/train/images/',
    'val': '/content/data/val/images',
    'names': ['IronCan_NoDamNoDir', 'IronCan_NoDamDir', 'IronCan_DamNoDir', 'IronCan_DamDir',
              'AluminumCan_NoDamNoDir', 'AluminumCan_NoDamDir', 'AluminumCan_DamNoDir', 'AluminumCan_DamDir',
              'Paper_NoDamNoDir', 'Paper_NoDamDir', 'Paper_DamNoDir', 'Paper_DamDir',
              'PetNoColor_NoDamNoDir', 'PetNoColor_NoDamDir', 'PetNoColor_DamNoDir', 'PetNoColor_DamDir',
              'PetColor_NoDamNoDir', 'PetColor_NoDamDir', 'PetColor_DamNoDir', 'PetColor_DamDir',
              'PE_NoDamNoDir', 'PE_NoDamDir', 'PE_DamNoDir', 'PE_DamDir',
              'PP_NoDamNoDir', 'PP_NoDamDir', 'PP_DamNoDir', 'PP_DamDir',
              'PS_NoDamNoDir', 'PS_NoDamDir', 'PS_DamNoDir', 'PS_DamDir',
              'Styrofoam_NoDamNoDir', 'Styrofoam_NoDamDir', 'Styrofoam_DamNoDir', 'Styrofoam_DamDir',
              'Vinyl_NoDamNoDir', 'Vinyl_NoDamDir', 'Vinyl_DamNoDir', 'Vinyl_DamDir',
              'GlassBrown_NoDamNoDir', 'GlassBrown_NoDamDir', 'GlassBrown_DamDir',
              'GlassGreen_NoDamNoDir', 'GlassGreen_NoDamDir', 'GlassGreen_DamDir',
              'GlassTransparency_NoDamNoDir', 'GlassTransparency_NoDamDir', 'GlassTransparency_DamDir'],
    'nc': 49
}

yaml_name = 'recycle_data.yaml'
with open('/content/data/' + 'recycle_data.yaml', 'w') as f:
  yaml.dump(data, f)

with open('/content/data/' + 'recycle_data.yaml', 'r') as f:
  recycle_yaml = yaml.safe_load(f)
  display(recycle_yaml)

{'names': ['IronCan_NoDamNoDir',
  'IronCan_NoDamDir',
  'IronCan_DamNoDir',
  'IronCan_DamDir',
  'AluminumCan_NoDamNoDir',
  'AluminumCan_NoDamDir',
  'AluminumCan_DamNoDir',
  'AluminumCan_DamDir',
  'Paper_NoDamNoDir',
  'Paper_NoDamDir',
  'Paper_DamNoDir',
  'Paper_DamDir',
  'PetNoColor_NoDamNoDir',
  'PetNoColor_NoDamDir',
  'PetNoColor_DamNoDir',
  'PetNoColor_DamDir',
  'PetColor_NoDamNoDir',
  'PetColor_NoDamDir',
  'PetColor_DamNoDir',
  'PetColor_DamDir',
  'PE_NoDamNoDir',
  'PE_NoDamDir',
  'PE_DamNoDir',
  'PE_DamDir',
  'PP_NoDamNoDir',
  'PP_NoDamDir',
  'PP_DamNoDir',
  'PP_DamDir',
  'PS_NoDamNoDir',
  'PS_NoDamDir',
  'PS_DamNoDir',
  'PS_DamDir',
  'Styrofoam_NoDamNoDir',
  'Styrofoam_NoDamDir',
  'Styrofoam_DamNoDir',
  'Styrofoam_DamDir',
  'Vinyl_NoDamNoDir',
  'Vinyl_NoDamDir',
  'Vinyl_DamNoDir',
  'Vinyl_DamDir',
  'GlassBrown_NoDamNoDir',
  'GlassBrown_NoDamDir',
  'GlassBrown_DamDir',
  'GlassGreen_NoDamNoDir',
  'GlassGreen_NoDamDir',
  'GlassGreen_DamDir

In [6]:
# YOLO 학습 위한 환경

%pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.165 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.3/78.2 GB disk)


In [7]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [8]:
# YOLO 전이 학습
from ultralytics import YOLO
import shutil, os

model_num = 0

# model = YOLO('yolov8m.pt')

# model.train(data='/content/data/recycle_data.yaml', epochs=100, patience=100, batch=16, imgsz=640,
#             name='recycle' + str(model_num), project='/content/drive/MyDrive/KCC_2nd_project/models_v4/', exist_ok=True)

In [9]:
# 연결이 끊기면 끊긴 곳에서부터 재학습

from ultralytics import YOLO
import shutil, os

model_num = 0

model = YOLO('/content/drive/MyDrive/KCC_2nd_project/models_v4/recycle' + str(model_num) + '/weights/last.pt')

model.train(resume=1)

Ultralytics YOLOv8.0.165 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/KCC_2nd_project/models_v4/recycle0/weights/last.pt, data=/content/data/recycle_data.yaml, epochs=100, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/KCC_2nd_project/models_v4/, name=recycle0, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, a

FailedPreconditionError: ignored

In [ ]:
# 예측 값 확인

from ultralytics import YOLO
import shutil, os
from PIL import Image
import matplotlib.pyplot as plt

model_num = 0

model = YOLO('/content/drive/MyDrive/KCC_2nd_project/models_v2/recycle'+ str(model_num) + '/weights/best.pt')

for i in range(10):
  results = model('/content/data/val/images/' + os.listdir('/content/data/val/images/')[i])

  # Show the results
  for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    plt.imshow(im)
    plt.show()